 1 Хотим взять информацию из группу Я люблю кино (на фото), но возможно поменяем в последствии на аналогичную (на данный момент берём именно её)  
 2 На данный момент хотим проанализировать количество лайков, просмотров и комментариев в зависимости от жанра фильма. Возможно создадим какой-нибудь индекс (например, лайки делить на просмотры), но пока сложно сказать точно  
 3 Целевая: количество лайков; Независимая: год выпуска, жанр, список главных ролей;  
   
 Гипотеза: больше лайков, собирают посты про фильмы жанра комедии, возрастное ограничение

In [388]:
import vk_api
def auth_handler():
    key = input("Authentication code: ")
    return key, False
login = input('Введите свой номер телефона для аккаунта ВКонтакте БЕЗ плюса в начале в формате 749512312312\n')
password = input("""Введите свой пароль к аккаунту ВКонтакте
                 (не волнуйтесь, мы его никуда не записываем - можете убедиться по ниже написанному коду)
                 """)
print('Вам нужно будет ввести код авторизации, который вам прислали во ВКонтакте')
vk_session = vk_api.VkApi(login, password, auth_handler=auth_handler, app_id=6287487, client_secret="QbYic1K3lEV5kTGiqlq2")
vk_session.auth()
vk = vk_session.get_api()

Введите свой номер телефона для аккаунта ВКонтакте БЕЗ плюса в начале в формате 749512312312
79687030122
Введите свой пароль к аккаунту ВКонтакте
                 (не волнуйтесь, мы его никуда не записываем - можете убедиться по ниже написанному коду)
                 Sha1tanMa1dan
Вам нужно будет ввести код авторизации, который вам прислали во ВКонтакте


KeyboardInterrupt: Interrupted by user

In [232]:
tools = vk_api.VkTools(vk)
wall_id = -113071474
wall = tools.get_all('wall.get', 4, {'owner_id': wall_id}, limit = 15000)


In [380]:
import re
from datetime import datetime

posts = wall['items']
posts = [post for post in posts if 'is_pinned' not in post.keys()]

posts = [post for post in posts if post['marked_as_ads'] == 0]

posts = [post for post in posts if post['owner_id'] == -113071474]

posts = [post for post in posts if len(re.findall('\([0-9]+\)', post['text'])) != 0]

not_posts = [post for post in posts if 'х:' not in post['text']]
not_posts = [post for post in not_posts if 'x:' not in post['text']]

posts = [
    post for post in posts if (
        'х:\n' in post['text'] or 'x:\n' in post['text'] 
            or 
        'х: \n' in post['text'] or 'x: \n' in post['text']
    )
]
print(len(posts))

1682


In [383]:
actors = [re.split('х:\n|x:\n|х: \n|x: \n', post['text']) for post in posts]
actors = [actor[1].replace('\n', '  ').replace('P.S.', ' ') for actor in actors]
actors = [actor.replace('– ', '- ').replace('— ', '- ').split('- ') for actor in actors]
for i in range(len(actors)):
    if len(actors[i]) == 1:
        actors[i] = actors[i][0].split('  ')
        if len(actors[i]) <= 1:
            actors[i] = ''
            continue
    
    actors[i] = [j.replace('  ', '') for j in actors[i] if j != '' and len(j) < 35 and '🏆' not in j and 'vk.' not in j]
    if actors[i] == []:
        actors[i] == ''
            
print(len(actors))

1682


In [365]:
for i in range(len(actors)):
    for j in range(len(actors[i])):
        if 'мoжeм повторит' in actors[i][j]:
            print(actors[i])
            print(actors[i][-1].lower())
            print(actors[i][-1].lower() == actors[i][-1])

In [338]:
wall['items'][0].keys()

dict_keys(['inner_type', 'ads_easy_promote', 'is_pinned', 'comments', 'marked_as_ads', 'activity', 'compact_attachments_before_cut', 'hash', 'type', 'donut_miniapp_url', 'attachments', 'attachments_meta', 'date', 'from_id', 'id', 'is_favorite', 'likes', 'owner_id', 'post_source', 'post_type', 'reposts', 'text', 'views'])

In [128]:
wall['items'][0]['owner_id']

-113071474

In [266]:
import pandas as pd
df = pd.DataFrame()

In [ ]:
df.index = [post['id'] for post in posts]
df['date'] = [post['date'] for post in posts]
df['date'] = pd.to_datetime(df['date'], unit='s')
df['date'] = df['date'].dt.date
col = ['likes', 'reposts', 'comments', 'views']
for c in col:
  df[c] = [post[c]['count'] for post in posts]
df['attachments'] = [len(post['attachments']) for post in posts]
df['text'] = [post['text'] for post in posts]
df = df.loc[df['text'] != '⚡⚡⚡']
df = df.loc[df['text'] != '']
df=df.dropna()
df = df.drop(index=45432794)
df = df.drop(index=44731519)
df['likes'].median(), df['likes'].mean()


In [384]:
import pandas as pd
df = pd.DataFrame()

df.index = list(map(lambda post: post.get('id', None), posts))
df['photos'] = list(map(lambda post: len(post['attachments']), posts))
                         
df['likes'] = list(map(lambda post: post.get('likes', {'count': None}).get('count', None), posts))
df['reposts'] = list(map(lambda post: post.get('reposts', {'count': None}).get('count', None), posts))
df['comments'] = list(map(lambda post: post.get('comments', {'count': None}).get('count', None), posts))
df['views'] = list(map(lambda post: post.get('views', {'count': None}).get('count', None), posts))
df['text'] = list(map(lambda post: post.get('text', None), posts))
df['actors'] = actors

df = df.loc[df['actors'] != '']

In [385]:
df

,photos,likes,reposts,comments,views,text,actors
4654254,4,185,447,5,63086,"Гoрод свoдит с умa, нo пoкинyть eго невoзмoжнo...","[Флoрeнс Пью, Гарpи Стайлc, Kpиc Пaйн, Oливия ..."
4654217,4,255,375,9,65356,"Тyт вeликoлeпный aктepcкий соcтaв Бpэд Питт, C...","[Caндpa Бyллoк, Чeннинг Taтум, Дэниэл Pэдклифф..."
4654151,4,215,150,13,75235,Гоcподи вcевышний 😱 Kaк жe кpутой фильм. Xищни...,"[Эмбeр Mидфандер , Дaкoта Биверc , Дэйн ДиЛьeг..."
4654127,4,233,349,11,76474,Oфигенный мyльт 👍 Пo жecтoкоcти - кaк в мульта...,"[Мaйя Эpскин, Джopдж Тaкэй, Maси Oкa, Кэри-Хиp..."
4654065,4,928,934,27,147536,Такоe кинo yже нe cнимают! 718 минyт нeопиcуeм...,"[Элaйджa Byд , Иэн Mаккeллен , Шoн Эcтин , Bиг..."
...,...,...,...,...,...,...,...
4439598,4,1587,3766,182,689506,Юнaя Екa|терина lI пытaeтся oбyстpoить Росcию....,[Эль Фaннинг]
4438234,4,2273,1991,53,558443,"Вoт тaкие фильмы нaдo cмoтpеть, чтoбы люди про...","[Хаpриcoн Фоpд, Oмаp Cи, Kаpa Ги, Дэн Cтивeнc]"
4438078,4,4673,11177,71,1183127,"Гoрoд cводит с умa, нo пoкинyть eгo нeвoзможнo...","[Флорeнс Пью, Кpис Пaйн, Oливия Уaйлд, Джeммa ..."
4435205,4,1616,2489,40,503924,"Пьянчyга и любитeль пyтaн, поcлe смеpти oтцa и...","[Tимоти Шaламe, Джoэл Эдгеpтoн, Poбеpт Пaттинc..."
